# Synthetic Expansion of Water Quality Dataset (35,000 rows)

This notebook loads `WQD.xlsx` and generates a **synthetic expanded dataset** (35,000 rows) that follows the **same column pattern** and approximately preserves **per-class distributions and correlations** using a **class-conditional Gaussian copula** approach.

Outputs:
- `WQD_synthetic_35000.xlsx`
- `WQD_synthetic_35000.csv`


In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# --- Settings ---
IN_PATH = "WQD.xlsx"             # change if needed
TARGET_N = 35000                 # desired number of rows
LABEL_COL = "Water Quality"
SEED = 42

# --- Load ---
df = pd.read_excel(IN_PATH)
feature_cols = [c for c in df.columns if c != LABEL_COL]

# Preserve original label distribution
counts = df[LABEL_COL].value_counts().sort_index()
proportions = (counts / counts.sum()).to_dict()
labels = list(counts.index)

target_counts = {lab: int(round(proportions[lab] * TARGET_N)) for lab in labels}
drift = TARGET_N - sum(target_counts.values())
if drift != 0:
    largest = max(target_counts, key=target_counts.get)
    target_counts[largest] += drift

def nearest_pd_corr(corr: np.ndarray, eps: float = 1e-8) -> np.ndarray:
    corr = (corr + corr.T) / 2.0
    w, v = np.linalg.eigh(corr)
    w = np.clip(w, eps, None)
    corr_pd = (v @ np.diag(w) @ v.T)
    d = np.sqrt(np.diag(corr_pd))
    corr_pd = corr_pd / np.outer(d, d)
    corr_pd = (corr_pd + corr_pd.T) / 2.0
    np.fill_diagonal(corr_pd, 1.0)
    return corr_pd

def empirical_quantile(values: np.ndarray, u: np.ndarray) -> np.ndarray:
    u = np.clip(u, 0.0, 1.0)
    return np.quantile(values, u, method="linear")

def gaussian_copula_sample_class(df_class: pd.DataFrame, n_samples: int, cols: list[str], seed: int) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    n = len(df_class)
    out = pd.DataFrame(index=range(n_samples), columns=cols, dtype=float)

    Z = np.zeros((n, len(cols)), dtype=float)
    col_values = {}
    col_minmax = {}
    for j, c in enumerate(cols):
        x = df_class[c].astype(float).to_numpy()
        col_values[c] = x
        col_minmax[c] = (np.nanmin(x), np.nanmax(x))

        r = pd.Series(x).rank(method="average").to_numpy()
        u = (r - 0.5) / n
        u = np.clip(u, 1e-6, 1 - 1e-6)
        Z[:, j] = norm.ppf(u)

    corr = np.corrcoef(Z, rowvar=False)
    corr = np.nan_to_num(corr, nan=0.0, posinf=0.0, neginf=0.0)
    corr = nearest_pd_corr(corr)

    z_new = rng.multivariate_normal(mean=np.zeros(len(cols)), cov=corr, size=n_samples, method="cholesky")
    u_new = norm.cdf(z_new)

    for j, c in enumerate(cols):
        x_new = empirical_quantile(col_values[c], u_new[:, j])
        mn, mx = col_minmax[c]
        out[c] = np.clip(x_new, mn, mx)

    return out

parts = []
for i, lab in enumerate(labels):
    df_class = df[df[LABEL_COL] == lab][feature_cols].reset_index(drop=True)
    n_lab = target_counts[lab]
    sampled = gaussian_copula_sample_class(df_class, n_lab, feature_cols, seed=SEED + i)
    sampled[LABEL_COL] = lab
    parts.append(sampled)

syn = pd.concat(parts, ignore_index=True)

# Shuffle and round for readability
syn = syn.sample(frac=1.0, random_state=123).reset_index(drop=True)
for c in feature_cols:
    syn[c] = syn[c].astype(float).round(6)

# --- Save ---
syn.to_excel("WQD_synthetic_35000.xlsx", index=False)
syn.to_csv("WQD_synthetic_35000.csv", index=False)

print("Original shape:", df.shape)
print("Synthetic shape:", syn.shape)
print("\nLabel distribution (synthetic):")
print(syn[LABEL_COL].value_counts(normalize=True).sort_index().mul(100).round(2).astype(str) + "%")

syn.head()

Original shape: (4300, 15)
Synthetic shape: (35000, 15)

Label distribution (synthetic):
Water Quality
0    32.56%
1    32.56%
2    34.89%
Name: proportion, dtype: object


,Temp,Turbidity (cm),DO(mg/L),BOD (mg/L),CO2,pH`,Alkalinity (mg L-1 ),Hardness (mg L-1 ),Calcium (mg L-1 ),Ammonia (mg L-1 ),Nitrite (mg L-1 ),Phosphorus (mg L-1 ),H2S (mg L-1 ),Plankton (No. L-1),Water Quality
0,29.227103,61.713178,3.594848,1.032362,7.976345,7.566291,75.292575,87.007520,33.997688,0.022152,0.007450,0.513808,0.019601,3742.451752,0
1,28.482818,67.537688,3.503741,1.769808,6.308790,7.257483,78.051200,141.064537,74.902467,0.023868,0.016039,1.901014,0.019398,3038.875971,0
2,34.013403,29.023695,6.604400,3.620926,2.591148,9.178917,29.985275,38.630780,115.615775,0.028983,0.671165,2.237975,0.018864,2853.550808,1
3,18.203761,17.951065,7.358075,3.735434,1.912308,6.386141,38.296150,178.742661,196.769854,0.049802,0.473423,2.101557,0.001494,2169.794658,1
4,26.124441,66.256173,4.808418,1.414935,6.671474,8.229495,49.171578,146.521790,51.854246,0.012241,0.019290,0.128071,0.019526,3359.172779,0
